# Chapter 6: Miller Indices and Crystal Planes

## Learning Objectives
- Understand Miller indices notation (hkl)
- Calculate and visualize crystal planes
- Compute d-spacing between planes
- Work with crystallographic directions [uvw]

---

## 6.1 What Are Miller Indices?

Miller indices provide a compact notation for describing:
1. **Crystal planes** (hkl)
2. **Crystal directions** [uvw]

### Defining a Plane (hkl)

To find Miller indices for a plane:
1. Find intercepts with crystal axes: $a/p$, $b/q$, $c/r$
2. Take reciprocals: $1/p$, $1/q$, $1/r$
3. Clear fractions to get smallest integers: $(h, k, l)$

**Examples:**
- Plane intercepting at (1, ∞, ∞) → (1, 0, 0) → **{100}**
- Plane intercepting at (1, 1, ∞) → (1, 1, 0) → **{110}**
- Plane intercepting at (1, 1, 1) → **{111}**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from typing import Tuple, List, Optional
from dataclasses import dataclass
import math

@dataclass
class LatticeParameters:
    """Lattice parameters for a crystal."""
    a: float
    b: float
    c: float
    alpha: float = 90.0  # degrees
    beta: float = 90.0
    gamma: float = 90.0

def lattice_vectors_from_parameters(params: LatticeParameters) -> np.ndarray:
    """Convert lattice parameters to Cartesian vectors."""
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    a_vec = np.array([params.a, 0, 0])
    bx = params.b * np.cos(gamma)
    by = params.b * np.sin(gamma)
    b_vec = np.array([bx, by, 0])
    
    cx = params.c * np.cos(beta)
    cy = params.c * (np.cos(alpha) - np.cos(beta) * np.cos(gamma)) / np.sin(gamma)
    cz = np.sqrt(max(0, params.c**2 - cx**2 - cy**2))
    c_vec = np.array([cx, cy, cz])
    
    return np.array([a_vec, b_vec, c_vec])

print("Lattice utilities loaded!")

## 6.2 Miller Indices for Cubic Systems

For cubic crystals, Miller indices are particularly simple because all axes are equivalent.

In [ ]:
class MillerIndex:
    """Represents Miller indices (hkl) for a crystal plane."""
    
    def __init__(self, h: int, k: int, l: int):
        """Initialize Miller indices.
        
        Args:
            h, k, l: Miller indices (will be reduced to lowest terms)
        """
        # Reduce to lowest terms
        gcd = math.gcd(math.gcd(abs(h) if h != 0 else 1, 
                                abs(k) if k != 0 else 1), 
                       abs(l) if l != 0 else 1)
        if gcd == 0:
            gcd = 1
        self.h = h // gcd if gcd > 0 else h
        self.k = k // gcd if gcd > 0 else k
        self.l = l // gcd if gcd > 0 else l
    
    @property
    def indices(self) -> Tuple[int, int, int]:
        """Return (h, k, l) tuple."""
        return (self.h, self.k, self.l)
    
    def normal_vector_cubic(self) -> np.ndarray:
        """Get normal vector to the plane (valid for cubic systems).
        
        For cubic systems, the plane (hkl) is perpendicular to [hkl].
        """
        n = np.array([self.h, self.k, self.l], dtype=float)
        return n / np.linalg.norm(n) if np.linalg.norm(n) > 0 else n
    
    def normal_vector_general(self, lattice_vectors: np.ndarray) -> np.ndarray:
        """Get normal vector for general crystal systems.
        
        Uses reciprocal lattice vectors: G = h*a* + k*b* + l*c*
        The plane (hkl) is perpendicular to G.
        """
        # Calculate reciprocal lattice vectors
        a, b, c = lattice_vectors
        V = np.dot(a, np.cross(b, c))
        a_star = 2 * np.pi * np.cross(b, c) / V
        b_star = 2 * np.pi * np.cross(c, a) / V
        c_star = 2 * np.pi * np.cross(a, b) / V
        
        G = self.h * a_star + self.k * b_star + self.l * c_star
        return G / np.linalg.norm(G)
    
    def __repr__(self):
        # Use bar notation for negative indices
        def fmt(x):
            if x < 0:
                return f"\u0305{abs(x)}"  # Overline for negative
            return str(x)
        return f"({fmt(self.h)}{fmt(self.k)}{fmt(self.l)})"

# Examples
planes = [
    MillerIndex(1, 0, 0),
    MillerIndex(1, 1, 0),
    MillerIndex(1, 1, 1),
    MillerIndex(2, 1, 1),
    MillerIndex(-1, 1, 0),
]

for plane in planes:
    print(f"{plane}: normal = {plane.normal_vector_cubic()}")

## 6.3 d-Spacing: Distance Between Planes

The **d-spacing** $d_{hkl}$ is the perpendicular distance between adjacent (hkl) planes.

### For Cubic Systems:
$$d_{hkl} = \frac{a}{\sqrt{h^2 + k^2 + l^2}}$$

### For General Systems:
$$d_{hkl} = \frac{2\pi}{|\mathbf{G}_{hkl}|}$$

where $\mathbf{G}_{hkl} = h\mathbf{a}^* + k\mathbf{b}^* + l\mathbf{c}^*$ is the reciprocal lattice vector.

In [ ]:
def d_spacing_cubic(h: int, k: int, l: int, a: float) -> float:
    """Calculate d-spacing for cubic crystal.
    
    Args:
        h, k, l: Miller indices
        a: Lattice constant in Å
    
    Returns:
        d-spacing in Å
    """
    sum_sq = h**2 + k**2 + l**2
    if sum_sq == 0:
        return np.inf
    return a / np.sqrt(sum_sq)

def d_spacing_general(h: int, k: int, l: int, params: LatticeParameters) -> float:
    """Calculate d-spacing for general crystal system.
    
    Uses the general formula involving the metric tensor.
    """
    alpha = np.radians(params.alpha)
    beta = np.radians(params.beta)
    gamma = np.radians(params.gamma)
    
    # Volume squared for triclinic
    cos_a, cos_b, cos_g = np.cos(alpha), np.cos(beta), np.cos(gamma)
    sin_a, sin_b, sin_g = np.sin(alpha), np.sin(beta), np.sin(gamma)
    
    V_sq = (params.a * params.b * params.c)**2 * (
        1 - cos_a**2 - cos_b**2 - cos_g**2 + 2*cos_a*cos_b*cos_g
    )
    V = np.sqrt(V_sq)
    
    # Reciprocal lattice calculation
    # 1/d^2 = (1/V^2) * [...]
    s11 = (params.b * params.c * sin_a)**2
    s22 = (params.a * params.c * sin_b)**2
    s33 = (params.a * params.b * sin_g)**2
    s12 = params.a * params.b * params.c**2 * (cos_a * cos_b - cos_g)
    s23 = params.a**2 * params.b * params.c * (cos_b * cos_g - cos_a)
    s13 = params.a * params.b**2 * params.c * (cos_g * cos_a - cos_b)
    
    inv_d_sq = (h**2 * s11 + k**2 * s22 + l**2 * s33 +
                2*h*k * s12 + 2*k*l * s23 + 2*h*l * s13) / V_sq
    
    return 1.0 / np.sqrt(inv_d_sq)

# Example: Silicon (diamond cubic, a = 5.431 Å)
a_Si = 5.431
common_planes = [(1,0,0), (1,1,0), (1,1,1), (2,0,0), (2,2,0), (3,1,1)]

print("Silicon d-spacings:")
print("═" * 30)
for h, k, l in common_planes:
    d = d_spacing_cubic(h, k, l, a_Si)
    print(f"d({h}{k}{l}) = {d:.4f} Å")

## 6.4 Visualizing Crystal Planes

A plane (hkl) intercepts the axes at positions $a/h$, $b/k$, $c/l$ (when indices are non-zero).

In [ ]:
def plot_miller_plane(h: int, k: int, l: int, a: float = 1.0,
                       lattice_vectors: np.ndarray = None) -> None:
    """Visualize a Miller plane in a unit cell."""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Use simple cubic if no vectors provided
    if lattice_vectors is None:
        lattice_vectors = np.eye(3) * a
    
    av, bv, cv = lattice_vectors
    
    # Draw unit cell
    origin = np.zeros(3)
    verts = [origin, av, av+bv, bv, cv, av+cv, av+bv+cv, bv+cv]
    edges = [(0,1), (1,2), (2,3), (3,0),
             (4,5), (5,6), (6,7), (7,4),
             (0,4), (1,5), (2,6), (3,7)]
    
    for i, j in edges:
        ax.plot3D(*zip(verts[i], verts[j]), 'b-', alpha=0.5)
    
    # Calculate intercepts (handle zero indices)
    intercepts = []
    labels = []
    
    if h != 0:
        intercepts.append(av / h)
        labels.append(f'a/{h}')
    if k != 0:
        intercepts.append(bv / k)
        labels.append(f'b/{k}')
    if l != 0:
        intercepts.append(cv / l)
        labels.append(f'c/{l}')
    
    # Mark intercept points
    intercepts = np.array(intercepts)
    ax.scatter(intercepts[:, 0], intercepts[:, 1], intercepts[:, 2],
               s=100, c='red', marker='o', label='Intercepts')
    
    for pt, lbl in zip(intercepts, labels):
        ax.text(pt[0], pt[1], pt[2], f'  {lbl}', fontsize=12)
    
    # Draw the plane
    if len(intercepts) >= 3:
        # Create triangular patch for 3 intercepts
        from mpl_toolkits.mplot3d.art3d import Poly3DCollection
        
        # For planes parallel to an axis, extend the polygon
        verts_plane = [intercepts.tolist()]
        poly = Poly3DCollection(verts_plane, alpha=0.4, 
                                  facecolor='green', edgecolor='darkgreen')
        ax.add_collection3d(poly)
    elif len(intercepts) == 2:
        # Plane parallel to one axis - need to extend
        # Find which axis is parallel
        if h == 0:
            parallel_vec = av
        elif k == 0:
            parallel_vec = bv
        else:
            parallel_vec = cv
        
        from mpl_toolkits.mplot3d.art3d import Poly3DCollection
        quad = np.array([
            intercepts[0],
            intercepts[1],
            intercepts[1] + parallel_vec,
            intercepts[0] + parallel_vec
        ])
        poly = Poly3DCollection([quad], alpha=0.4,
                                  facecolor='green', edgecolor='darkgreen')
        ax.add_collection3d(poly)
    
    # Draw normal vector
    miller = MillerIndex(h, k, l)
    normal = miller.normal_vector_general(lattice_vectors)
    center = np.mean(intercepts, axis=0)
    
    ax.quiver(center[0], center[1], center[2],
              normal[0], normal[1], normal[2],
              length=0.5*a, color='purple', arrow_length_ratio=0.2,
              linewidth=2, label='Normal')
    
    # Labels
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'Miller Plane {miller}', fontsize=14)
    ax.legend()
    
    # Equal aspect ratio
    max_range = a * 1.2
    ax.set_xlim([0, max_range])
    ax.set_ylim([0, max_range])
    ax.set_zlim([0, max_range])
    
    plt.tight_layout()
    return fig, ax

# Visualize common planes
planes_to_plot = [(1,0,0), (1,1,0), (1,1,1)]

for h, k, l in planes_to_plot:
    plot_miller_plane(h, k, l, a=1.0)
    plt.show()

## 6.5 Families of Planes and Multiplicity

In cubic crystals, certain planes are **equivalent by symmetry**:
- {100} includes (100), (010), (001), (̄100), (0̄10), (00̄1) — 6 planes
- {110} includes 12 equivalent planes
- {111} includes 8 equivalent planes

In [ ]:
def equivalent_planes_cubic(h: int, k: int, l: int) -> List[Tuple[int, int, int]]:
    """Generate all symmetry-equivalent planes for cubic crystal.
    
    Returns all permutations and sign changes of (h, k, l).
    """
    from itertools import permutations, product
    
    indices = [h, k, l]
    equivalent = set()
    
    # All permutations
    for perm in permutations(indices):
        # All sign combinations
        for signs in product([1, -1], repeat=3):
            plane = tuple(s * p for s, p in zip(signs, perm))
            equivalent.add(plane)
    
    return sorted(list(equivalent))

# Example: multiplicity of common planes
test_planes = [(1,0,0), (1,1,0), (1,1,1), (2,1,0), (2,1,1)]

print("Plane Multiplicity in Cubic Systems:")
print("═" * 40)
for h, k, l in test_planes:
    equiv = equivalent_planes_cubic(h, k, l)
    print(f"{{h}{k}{l}}}: {len(equiv)} equivalent planes")
    if len(equiv) <= 12:
        print(f"  {equiv[:6]}")
        if len(equiv) > 6:
            print(f"  {equiv[6:]}")

## 6.6 Crystal Directions [uvw]

Crystallographic directions are denoted [uvw], representing the direction:
$$\mathbf{d} = u\mathbf{a} + v\mathbf{b} + w\mathbf{c}$$

**Important relationships for cubic systems:**
- Direction [uvw] is perpendicular to plane (uvw)
- Angle between directions can be calculated using the metric tensor

In [ ]:
class CrystalDirection:
    """Represents a crystallographic direction [uvw]."""
    
    def __init__(self, u: int, v: int, w: int):
        gcd = math.gcd(math.gcd(abs(u) if u != 0 else 1,
                                abs(v) if v != 0 else 1),
                       abs(w) if w != 0 else 1)
        if gcd == 0:
            gcd = 1
        self.u = u // gcd if gcd > 0 else u
        self.v = v // gcd if gcd > 0 else v
        self.w = w // gcd if gcd > 0 else w
    
    def to_cartesian(self, lattice_vectors: np.ndarray) -> np.ndarray:
        """Convert to Cartesian direction vector."""
        return self.u * lattice_vectors[0] + self.v * lattice_vectors[1] + self.w * lattice_vectors[2]
    
    def __repr__(self):
        return f"[{self.u}{self.v}{self.w}]"

def angle_between_directions(d1: CrystalDirection, d2: CrystalDirection,
                              lattice_vectors: np.ndarray) -> float:
    """Calculate angle between two crystal directions."""
    v1 = d1.to_cartesian(lattice_vectors)
    v2 = d2.to_cartesian(lattice_vectors)
    
    cos_angle = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
    return np.degrees(np.arccos(np.clip(cos_angle, -1, 1)))

def angle_between_planes(m1: MillerIndex, m2: MillerIndex,
                          lattice_vectors: np.ndarray) -> float:
    """Calculate angle between two crystal planes."""
    n1 = m1.normal_vector_general(lattice_vectors)
    n2 = m2.normal_vector_general(lattice_vectors)
    
    cos_angle = np.dot(n1, n2)
    return np.degrees(np.arccos(np.clip(cos_angle, -1, 1)))

# Examples: cubic crystal
cubic_vectors = np.eye(3) * 4.0  # Cubic with a = 4 Å

# Angles between directions
dir_100 = CrystalDirection(1, 0, 0)
dir_110 = CrystalDirection(1, 1, 0)
dir_111 = CrystalDirection(1, 1, 1)

print("Angles between directions (cubic):")
print(f"∠({dir_100}, {dir_110}) = {angle_between_directions(dir_100, dir_110, cubic_vectors):.1f}°")
print(f"∠({dir_100}, {dir_111}) = {angle_between_directions(dir_100, dir_111, cubic_vectors):.1f}°")
print(f"∠({dir_110}, {dir_111}) = {angle_between_directions(dir_110, dir_111, cubic_vectors):.1f}°")

# Verify: angles between planes
plane_100 = MillerIndex(1, 0, 0)
plane_110 = MillerIndex(1, 1, 0)
plane_111 = MillerIndex(1, 1, 1)

print("\nAngles between planes (cubic):")
print(f"∠{plane_100} ↔ {plane_110} = {angle_between_planes(plane_100, plane_110, cubic_vectors):.1f}°")
print(f"∠{plane_100} ↔ {plane_111} = {angle_between_planes(plane_100, plane_111, cubic_vectors):.1f}°")
print(f"∠{plane_110} ↔ {plane_111} = {angle_between_planes(plane_110, plane_111, cubic_vectors):.1f}°")

## 6.7 Systematic Absences (Extinction Rules)

Not all (hkl) planes produce diffraction peaks. **Systematic absences** occur due to:

1. **Lattice centering:**
   - FCC: Reflections only when h, k, l all odd or all even
   - BCC: Reflections only when h + k + l = even

2. **Screw axes and glide planes** (in space groups)

In [ ]:
def is_allowed_reflection(h: int, k: int, l: int, structure_type: str) -> bool:
    """Check if (hkl) reflection is allowed for a given structure.
    
    Args:
        h, k, l: Miller indices
        structure_type: 'sc' (simple cubic), 'bcc', 'fcc', or 'diamond'
    """
    structure_type = structure_type.lower()
    
    if structure_type == 'sc':
        return True  # All allowed
    
    elif structure_type == 'bcc':
        # h + k + l must be even
        return (h + k + l) % 2 == 0
    
    elif structure_type == 'fcc':
        # h, k, l must be all odd or all even
        parities = [h % 2, k % 2, l % 2]
        return len(set(parities)) == 1  # All same parity
    
    elif structure_type == 'diamond':
        # FCC rule + additional extinctions
        if not is_allowed_reflection(h, k, l, 'fcc'):
            return False
        # Additional diamond rule: if all even, h+k+l must be divisible by 4
        if h % 2 == 0 and k % 2 == 0 and l % 2 == 0:
            return (h + k + l) % 4 == 0
        return True
    
    else:
        raise ValueError(f"Unknown structure type: {structure_type}")

# Generate allowed reflections for different structures
def generate_reflections(max_index: int, structure_type: str) -> List[Tuple[int, int, int]]:
    """Generate all allowed reflections up to max_index."""
    reflections = []
    for h in range(max_index + 1):
        for k in range(h + 1):
            for l in range(k + 1):
                if h == k == l == 0:
                    continue
                if is_allowed_reflection(h, k, l, structure_type):
                    reflections.append((h, k, l))
    return reflections

# Compare reflection rules
structures = ['sc', 'bcc', 'fcc', 'diamond']

print("Allowed reflections (up to index 3):")
print("═" * 60)

for struct in structures:
    refs = generate_reflections(3, struct)
    print(f"\n{struct.upper()}:")
    print(f"  {refs}")

## 6.8 Zone Axis

A **zone** is a set of planes that share a common direction (the **zone axis**).

If plane (hkl) belongs to zone [uvw], then:
$$hu + kv + lw = 0$$

The zone axis of two planes $(h_1 k_1 l_1)$ and $(h_2 k_2 l_2)$ is:
$$[uvw] = (h_1 k_1 l_1) \times (h_2 k_2 l_2)$$

In [ ]:
def zone_axis(plane1: MillerIndex, plane2: MillerIndex) -> CrystalDirection:
    """Calculate the zone axis of two planes.
    
    The zone axis is perpendicular to both plane normals.
    """
    v1 = np.array([plane1.h, plane1.k, plane1.l])
    v2 = np.array([plane2.h, plane2.k, plane2.l])
    
    zone = np.cross(v1, v2)
    
    # Reduce to integers
    u, v, w = map(int, zone)
    return CrystalDirection(u, v, w)

def is_plane_in_zone(plane: MillerIndex, zone: CrystalDirection) -> bool:
    """Check if a plane belongs to a zone."""
    return (plane.h * zone.u + plane.k * zone.v + plane.l * zone.w) == 0

def planes_in_zone(zone: CrystalDirection, max_index: int = 3) -> List[MillerIndex]:
    """Find all planes belonging to a zone."""
    planes = []
    for h in range(-max_index, max_index + 1):
        for k in range(-max_index, max_index + 1):
            for l in range(-max_index, max_index + 1):
                if h == k == l == 0:
                    continue
                plane = MillerIndex(h, k, l)
                if is_plane_in_zone(plane, zone):
                    # Normalize to positive first index
                    if plane.h < 0 or (plane.h == 0 and plane.k < 0) or \
                       (plane.h == 0 and plane.k == 0 and plane.l < 0):
                        plane = MillerIndex(-plane.h, -plane.k, -plane.l)
                    planes.append(plane)
    
    # Remove duplicates
    unique = []
    seen = set()
    for p in planes:
        key = (p.h, p.k, p.l)
        if key not in seen:
            seen.add(key)
            unique.append(p)
    
    return unique

# Example: Find zone axis of (100) and (010)
p1 = MillerIndex(1, 0, 0)
p2 = MillerIndex(0, 1, 0)
zone = zone_axis(p1, p2)
print(f"Zone axis of {p1} and {p2}: {zone}")

# Find all planes in [001] zone
planes_001 = planes_in_zone(CrystalDirection(0, 0, 1))
print(f"\nPlanes in [001] zone: {planes_001}")

---

## Practice Exercises

### Exercise 6.1: Calculate d-spacings
Calculate the first 5 allowed d-spacings for FCC aluminum (a = 4.05 Å).

In [ ]:
# YOUR CODE HERE
a_Al = 4.05
# Hint: Use is_allowed_reflection() and d_spacing_cubic()

### Exercise 6.2: Hexagonal Miller-Bravais Indices

Hexagonal crystals often use 4-index notation (hkil) where i = -(h+k).
Write a function to convert between 3-index and 4-index notation.

In [ ]:
# YOUR CODE HERE
def hkl_to_hkil(h: int, k: int, l: int) -> Tuple[int, int, int, int]:
    """Convert 3-index to 4-index hexagonal notation."""
    # TODO: Implement
    pass

def hkil_to_hkl(h: int, k: int, i: int, l: int) -> Tuple[int, int, int]:
    """Convert 4-index to 3-index notation."""
    # TODO: Implement (verify that i = -(h+k))
    pass

### Exercise 6.3: Interplanar Angle Calculation

For a tetragonal crystal with a = 4 Å and c = 6 Å, calculate the angle between (101) and (011).

In [ ]:
# YOUR CODE HERE
# Hint: Use angle_between_planes() with appropriate lattice vectors

---

## Key Takeaways

1. **Miller indices (hkl)** are reciprocals of axis intercepts
2. **d-spacing** = perpendicular distance between planes; critical for diffraction
3. **Direction [uvw]** perpendicular to plane (uvw) only in cubic systems
4. **Systematic absences** due to centering help identify crystal structures
5. **Zone axis** is a direction common to multiple planes

## Next Chapter Preview

In Chapter 7, we'll use Miller indices to **cut surfaces and create slab models** for surface science simulations.